# Task1.Làm sạch product_id, product_name và category


In [2]:
import pandas as pd
import unicodedata
import re
def chuan_hoa_category(s):
    if pd.isna(s):
        return s
    s = str(s).lower().strip()
    s = unicodedata.normalize("NFC", s)
    s = " ".join(s.split())
    if "dien" in s or "điện" in s:
        return "điện tử"
    if 'gia dung' in s:
        return 'gia dụng'
    return s
def chuan_hoa_chuoi(s):
    if pd.isna(s):
        return s
    s=str(s).strip().lower()
    s=' '.join(s.split())
    return unicodedata.normalize("NFC",s)

def chuan_hoa_time(x):
    if pd.isna(x):
        return x
    x=str(x).strip()
    if re.match(r"^\d{4}[/-]\d{2}[/-]\d{2}$", x):
        return x.replace("/", "-")
    if re.match(r"^\d{2}/\d{2}/\d{4}$", x):
        d, m, y = x.split("/")
        return f"{y}-{m}-{d}"
    return x

def chuan_hoa_gia_tien(x):
    if pd.isna(x):
        return 0
    x = str(x).lower().strip()
    x = x.replace("đ", "")
    if "k" in x:
        so = re.sub(r"[^\d.]", "", x)
        try:
            return float(so) * 1000
        except:
            return 0
    x = re.sub(r"[.,](?=\d{3}\b)", "", x)
    try:
        return float(x)
    except:
        return 0

product_df=pd.read_csv('product_catalog.csv')
order_df=pd.read_csv('order_details.csv')
refund_df=pd.read_csv('return_refund.csv')
def chuan_hoa_id(s):
    if pd.isna(s):
        return s
    return str(s).strip().upper()

def chuan_hoa_quantity(x):
    if pd.isna(x):
        return None
    match = re.search(r'-?\d+', str(x))
    if match:
        return int(match.group())
    return None
def chuan_hoa_reason(s):
    if pd.isna(s):
        return "khác"
    s = str(s).lower().strip()
    s = re.sub(r'[^a-z\s]', '', s)
    s = " ".join(s.split())
    if "loi" in s or "lỗi" in s:
        return "lỗi sản phẩm"
    if "meo" in s or "méo" in s:
        return "méo mó"
    if "hong" in s or "hỏng" in s:
        return "hỏng"
    return "khác"

product_df['product_id']=product_df['product_id'].apply(chuan_hoa_id)
product_df['product_name']=product_df['product_name'].apply(chuan_hoa_chuoi)
product_df["category"] = product_df["category"].apply(chuan_hoa_category)
product_df.to_csv('product_catalog_clean.csv',index=False)

order_df['product_id']=order_df['product_id'].apply(chuan_hoa_id)
order_df['quantity']=order_df['quantity'].apply(chuan_hoa_quantity)
order_df['unit_price']=(
    order_df['unit_price']
    .astype(str)
    .str.replace(',','',regex=False)
)
order_df['unit_price']=order_df['unit_price'].apply(chuan_hoa_gia_tien)
order_df["order_date"] = order_df["order_date"].apply(chuan_hoa_time)
order_df.to_csv('order_details_clean.csv',index=False)

refund_df['product_id']=refund_df['product_id'].apply(chuan_hoa_id)


refund_df['refund_amount'] = pd.to_numeric(
    refund_df['refund_amount']
    .astype(str)
    .str.replace(r'[^\d.]', '', regex=True),
    errors='coerce'
).fillna(0)
refund_df['reason']=refund_df['reason'].apply(chuan_hoa_reason)

refund_df.to_csv('return_refund_clean.csv',index=False)
